In [5]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import GridSearchCV
dataset=pd.read_csv("diabetes.csv",index_col=None)

In [6]:
X=dataset[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age',]]
y=dataset['Outcome']
def selectpca(X_train,X_test,n):
        pca = PCA(n_components=n)
        pca = PCA(n_components = 5)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)
        explained_variance = pca.explained_variance_ratio_
        return X_train, X_test, explained_variance
#def split_scalar(indep_X,dep_Y): #train test split preprocess
def split_scalar(X,y, test_size=0.2, random_state=0):         
        #X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
def cm_prediction(grid, X_test, y_test):
 
        grid_predictions = grid.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, grid_predictions)     # classification algorithm ML using confusion matrix
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report
        
        Accuracy=accuracy_score(y_test, grid_predictions )        #using accuracy
        
        report=classification_report(y_test, grid_predictions)     #using report
        return  grid,Accuracy,report,grid_predictions,cm
def logistic(X_train,y_train):       
        # Fitting K-NN to the Training set
    from sklearn.linear_model import LogisticRegression
    param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 
    grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')         
    grid.fit(X_train, y_train)
    grid,Accuracy,report,grid_predictions,cm=cm_prediction(grid,X_test,y_test)   #cm_prediction called here
    return grid,Accuracy,report,grid_predictions,cm 
      
def selectk_Classification(acclog): 
    
    dataframe=pd.DataFrame(index=['PCA'],columns=['Logistic'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        
    return dataframe

In [7]:
acclog=[]
X_train, X_test, y_train, y_test=split_scalar(X,y, test_size=0.2, random_state=0)   
grid,Accuracy,report,grid_predictions,cm=logistic(X_train,y_train)
acclog.append(Accuracy)
result=selectk_Classification(acclog)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11996\970378758.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['Logistic'][idex]=acclog[number]


In [9]:
result
#5

,Logistic
PCA,0.824675


In [10]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8661761781666335

In [11]:
grid.predict([[5,70,40,20,1,47.6,0.345,11]])

array([1], dtype=int64)